In [ ]:
from google.colab import files
!pip install pyyaml h5py 
import keras
from keras.layers import LSTM, Dropout, Dense
import tensorflow as tf
from tensorflow.keras.utils import Sequence
import numpy as np
import pandas as pd
uploaded = files.upload()

Saving final_train_data.csv to final_train_data.csv


In [ ]:
data = pd.read_csv("final_train_data.csv")
data.shape

(319998, 16)

In [ ]:
xtrain=data.iloc[:,:15]
ytrain=data['label']

In [ ]:
ds = tf.keras.utils.timeseries_dataset_from_array(
      data=xtrain,
      targets=ytrain,
      sequence_length=720,
      sequence_stride=1,
      shuffle=True,
      batch_size=254)

In [ ]:
model = keras.Sequential()
model.add(LSTM(128, input_shape = (720, 15), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy"
              , metrics=[keras.metrics.binary_accuracy]
              , optimizer="adam")

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 720, 128)          73728     
                                                                 
 dropout (Dropout)           (None, 720, 128)          0         
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 205,441
Trainable params: 205,441
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(ds, batch_size=254,epochs=10)

Epoch 1/10
1258/1258 [==============================] - 199s 151ms/step - loss: 0.6805 - binary_accuracy: 0.5429
Epoch 2/10
1258/1258 [==============================] - 193s 153ms/step - loss: 0.6105 - binary_accuracy: 0.6261
Epoch 3/10
1258/1258 [==============================] - 193s 153ms/step - loss: 0.2745 - binary_accuracy: 0.8837
Epoch 4/10
1258/1258 [==============================] - 194s 154ms/step - loss: 0.0781 - binary_accuracy: 0.9711
Epoch 5/10
1258/1258 [==============================] - 196s 155ms/step - loss: 0.0202 - binary_accuracy: 0.9931
Epoch 6/10
1258/1258 [==============================] - 210s 167ms/step - loss: 0.0101 - binary_accuracy: 0.9969
Epoch 7/10
1258/1258 [==============================] - 194s 154ms/step - loss: 0.0060 - binary_accuracy: 0.9981
Epoch 8/10
1258/1258 [==============================] - 192s 153ms/step - loss: 0.0044 - binary_accuracy: 0.9986
Epoch 9/10
1258/1258 [==============================] - 193s 153ms/step - loss: 0.0049 - binary_

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving final_test_data.csv to final_test_data.csv


In [ ]:
test_data = pd.read_csv("final_test_data.csv")
test_data.shape

(96651, 16)

In [ ]:
xtest=test_data.iloc[:,:15]
ytest=test_data['label']

In [ ]:
ds_test = tf.keras.utils.timeseries_dataset_from_array(
      data=xtest,
      targets=ytest,
      sequence_length=720,
      sequence_stride=1,
      shuffle=True,
      batch_size=254)

In [ ]:
model.evaluate(ds_test)

378/378 [==============================] - 28s 71ms/step - loss: 1.3957 - binary_accuracy: 0.8348


[1.3957232236862183, 0.8347787857055664]

In [ ]:
class DatasetSequence(Sequence):

    def __init__(self, batch_size):
        self.data = pd.read_csv("data_bi1.csv")
        self.batch_size = batch_size

    def __len__(self):
        return self.data.shape[0]-self.batch_size-2000

    def generate_data(self,X, y,i,batch_size, sequence_length = 1500, step = 1):
      X_local = []
      y_local = []
      print(i)
      for start in range(i, i+batch_size, step):
        end = start + sequence_length
        X_local.append(X[start:end])
        y_local.append(y)
      return np.array(X_local), np.array(y_local)

      
    def __getitem__(self, idx):
        return self.generate_data(X=self.data,y=1,i=idx,batch_size=self.batch_size, sequence_length = 1500, step = 1)